# Getting started with ONNX IR 🌱
The ONNX IR ships with the ONNX Script package and is available as `onnx_ir`.
To create an IR object from ONNX file, load it as `ModelProto` and call
`ir.from_proto()`:

In [1]:
# Define an example model for this example
MODEL_TEXT = r"""
<
   ir_version: 8,
   opset_import: ["" : 18],
   producer_name: "pytorch",
   producer_version: "2.0.0"
>
torch_jit (float[5,5,5] input_0) => (float[5,5] val_19, float[5,5] val_6) {
   [node_1] val_1 = Constant <value_int: ints = [1]> ()
   [node_2] val_2 = Shape <start: int = 0> (val_1)
   [node_3] val_3 = Size (val_2)
   [node_4] val_4 = Constant <value: tensor = int64 {0}> ()
   [node_5] val_5 = Equal (val_3, val_4)
   [node_6] val_6 = ReduceMean <keepdims: int = 0, noop_with_empty_axes: int = 0> (input_0, val_1)
   [node_7] val_7 = ReduceMean <keepdims: int = 1, noop_with_empty_axes: int = 0> (input_0, val_1)
   [node_8] val_8 = Shape <start: int = 0> (input_0)
   [node_9] val_9 = Gather <axis: int = 0> (val_8, val_1)
   [node_10] val_10 = ReduceProd <keepdims: int = 0, noop_with_empty_axes: int = 0> (val_9)
   [node_11] val_11 = Sub (input_0, val_7)
   [node_12] val_12 = Mul (val_11, val_11)
   [node_13] val_13 = ReduceMean <keepdims: int = 0, noop_with_empty_axes: int = 0> (val_12, val_1)
   [node_14] val_14 = Cast <to: int = 1> (val_10)
   [node_15] val_15 = Mul (val_13, val_14)
   [node_16] val_16 = Constant <value: tensor = int64 {1}> ()
   [node_17] val_17 = Sub (val_10, val_16)
   [node_18] val_18 = Cast <to: int = 1> (val_17)
   [node_19] val_19 = Div (val_15, val_18)
}
"""

In [2]:
import onnx

import onnx_ir as ir

# Load the model as onnx.ModelProto
# You can also load the model from a file using onnx.load("model.onnx")
model_proto = onnx.parser.parse_model(MODEL_TEXT)

# Create an IR object from the model
model = ir.from_proto(model_proto)

Now we can explore the IR object

In [3]:
print(f"The main graph has {len(model.graph)} nodes.")

The main graph has 19 nodes.


All inputs

In [4]:
print(model.graph.inputs)

[Value(name='input_0', type=Tensor(FLOAT), shape=Shape([5, 5, 5]))]


All outputs

In [5]:
print(model.graph.outputs)

[Value(name='val_19', type=Tensor(FLOAT), shape=Shape([5, 5]), producer='node_19', index=0), Value(name='val_6', type=Tensor(FLOAT), shape=Shape([5, 5]), producer='node_6', index=0)]


Nodes that uses the first input

In [6]:
print(list(model.graph.inputs[0].uses()))

[Usage(node=Node(name='node_6', domain='', op_type='ReduceMean', inputs=(Value(name='input_0', type=Tensor(FLOAT), shape=Shape([5, 5, 5])), Value(name='val_1', producer='node_1', index=0)), attributes={'keepdims': Attr('keepdims', INT, 0), 'noop_with_empty_axes': Attr('noop_with_empty_axes', INT, 0)}, overload='', outputs=(Value(name='val_6', type=Tensor(FLOAT), shape=Shape([5, 5]), producer='node_6', index=0),), version=None, doc_string=None), idx=0), Usage(node=Node(name='node_7', domain='', op_type='ReduceMean', inputs=(Value(name='input_0', type=Tensor(FLOAT), shape=Shape([5, 5, 5])), Value(name='val_1', producer='node_1', index=0)), attributes={'keepdims': Attr('keepdims', INT, 1), 'noop_with_empty_axes': Attr('noop_with_empty_axes', INT, 0)}, overload='', outputs=(Value(name='val_7', producer='node_7', index=0),), version=None, doc_string=None), idx=0), Usage(node=Node(name='node_8', domain='', op_type='Shape', inputs=(Value(name='input_0', type=Tensor(FLOAT), shape=Shape([5, 5, 

The node that produces the last output (as the i-th output)

In [7]:
print(model.graph.outputs[-1].producer())
print(model.graph.outputs[-1].index())

%"val_6"<FLOAT,[5,5]> ⬅️ ::ReduceMean(%"input_0", %"val_1") {keepdims=0, noop_with_empty_axes=0}
0


Print the graph

In [8]:
print(model.graph)

graph(
    name=torch_jit,
    inputs=(
        %"input_0"<FLOAT,[5,5,5]>
    ),
    outputs=(
        %"val_19"<FLOAT,[5,5]>,
        %"val_6"<FLOAT,[5,5]>
    ),
) {
     0 |  # node_1
          %"val_1"<?,?> ⬅️ ::Constant() {value_int=(1,)}
     1 |  # node_2
          %"val_2"<?,?> ⬅️ ::Shape(%"val_1") {start=0}
     2 |  # node_3
          %"val_3"<?,?> ⬅️ ::Size(%"val_2")
     3 |  # node_4
          %"val_4"<?,?> ⬅️ ::Constant() {value=TensorProtoTensor<INT64,[]>(array(0), name='')}
     4 |  # node_5
          %"val_5"<?,?> ⬅️ ::Equal(%"val_3", %"val_4")
     5 |  # node_6
          %"val_6"<FLOAT,[5,5]> ⬅️ ::ReduceMean(%"input_0", %"val_1") {keepdims=0, noop_with_empty_axes=0}
     6 |  # node_7
          %"val_7"<?,?> ⬅️ ::ReduceMean(%"input_0", %"val_1") {keepdims=1, noop_with_empty_axes=0}
     7 |  # node_8
          %"val_8"<?,?> ⬅️ ::Shape(%"input_0") {start=0}
     8 |  # node_9
          %"val_9"<?,?> ⬅️ ::Gather(%"val_8", %"val_1") {axis=0}
     9 |  # node_10
       

Convert from the IR object back to ModelProto

In [9]:
model_proto_back = ir.to_proto(model)

## Next steps

Read the introductions for a more detailed introduction of the IR
(Documentation in progress 🚧)